In [2]:
from glob import glob
import os
import torch
import torchmetrics
import nibabel as nib
import subprocess
import numpy as np
import torchmetrics
import pandas as pd
region_names = ["bg", "hy", "mb", "hpf", "str", "ctx", "gp", "th", "cb", "olf", "bs"]

In [3]:
atlas_mask = "dataset3/Atlas/P56_Annotation_128_norm_id_mask.nii.gz"
atlas_labels = "atlas_gin_map6.nii.gz"

affine_labels = sorted(glob(os.path.join('dataset3', 'GIN', 'Labels', "*_affine_convert.nii.gz")))

deep1_labels = sorted(glob(os.path.join('output', 'GIN', 'scenario1', 'Labels', '*.nii.gz')))
deep2_labels = sorted(glob(os.path.join('output', 'GIN', 'scenario2', 'Labels', '*.nii.gz')))
deep3_labels = sorted(glob(os.path.join('output', 'GIN', 'scenario3', 'Labels', '*.nii.gz')))
ants_labels = sorted(glob(os.path.join('dataset3', 'GIN', 'Labels', "*_deformable_convert.nii.gz")))
mean_labels = sorted(glob(os.path.join('dataset3', 'GIN_Mean', 'Labels', "*_deformable_convert.nii.gz")))

deep1_fwd_warps = sorted(glob(os.path.join('output', 'GIN', 'scenario1', 'DeformableWarp', "*_warp.nii.gz")))
deep2_fwd_warps = sorted(glob(os.path.join('output', 'GIN', 'scenario2', 'DeformableWarp', "*_warp.nii.gz")))
deep3_fwd_warps = sorted(glob(os.path.join('output', 'GIN', 'scenario3', 'DeformableWarp', "*_warp.nii.gz")))
ants_fwd_warps = sorted(glob(os.path.join('dataset3', 'GIN', 'MRI', "*_warp.nii.gz")))
mean_fwd_warps = sorted(glob(os.path.join('dataset3', 'GIN_Mean', 'MRI', "*_warp.nii.gz")))

deep1_bck_warps = sorted(glob(os.path.join('output', 'GIN', 'scenario1', 'DeformableWarp', "*_invwarp.nii.gz")))
deep2_bck_warps = sorted(glob(os.path.join('output', 'GIN', 'scenario2', 'DeformableWarp', "*_invwarp.nii.gz")))
deep3_bck_warps = sorted(glob(os.path.join('output', 'GIN', 'scenario3', 'DeformableWarp', "*_invwarp.nii.gz")))
ants_bck_warps = sorted(glob(os.path.join('dataset3', 'GIN', 'MRI', "*_warp_inverse.nii.gz")))
mean_bck_warps = sorted(glob(os.path.join('dataset3', 'GIN_Mean', 'MRI', "*_warp_inverse.nii.gz")))

dicemetric = torchmetrics.Dice(average=None, mdmc_average='global', num_classes=11)

values_affine = [0,0,0,0,0,0,0,0,0,0,0]
values_deep1 = [0,0,0,0,0,0,0,0,0,0,0]
values_deep2 = [0,0,0,0,0,0,0,0,0,0,0]
values_deep3 = [0,0,0,0,0,0,0,0,0,0,0]
values_ants = [0,0,0,0,0,0,0,0,0,0,0]
values_mean = [0,0,0,0,0,0,0,0,0,0,0]

mapping_csv = "dataset3/GIN/labels_mapping.csv"
data = pd.read_csv(mapping_csv)
    
count = 0

list_dict = []

for i, _ in enumerate(ants_labels): 
    affine_ith_labels = torch.from_numpy(nib.load(affine_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
    deep1_ith_labels = torch.from_numpy(nib.load(deep1_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
    deep2_ith_labels = torch.from_numpy(nib.load(deep2_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
    deep3_ith_labels = torch.from_numpy(nib.load(deep3_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
    ants_ith_labels = torch.from_numpy(nib.load(ants_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
    mean_ith_labels = torch.from_numpy(nib.load(mean_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
    
    for j, _ in enumerate(ants_labels):
        if j == i:  
            print(str(i+1) + '_' + str(j+1), end='\r')
            count += 1            
            
            affine_jth_labels = torch.from_numpy(nib.load(affine_labels[j]).get_fdata().ravel()).type('torch.LongTensor')
            affine_values_ij = dicemetric(affine_jth_labels,affine_ith_labels)
            
            output = 'deep1_j_to_i_labels.nii.gz'
            command = ['antsApplyTransforms']
            command.extend(['-d', '3'])
            command.extend(['-i', deep1_labels[j]])
            command.extend(['-r', deep1_labels[i]])
            command.extend(['-o', output])
            command.extend(['-n', 'NearestNeighbor'])
            command.extend(['-t', deep1_bck_warps[i]])
            command.extend(['-t', deep1_fwd_warps[j]])
            subprocess.call(command)             
            deep1_jth_labels_to_ith_labels = torch.from_numpy(nib.load(output).get_fdata().ravel()).type('torch.LongTensor')
            deep1_values_ij = dicemetric(deep1_jth_labels_to_ith_labels,deep1_ith_labels)
            
            output = 'deep2_j_to_i_labels.nii.gz'
            command = ['antsApplyTransforms']
            command.extend(['-d', '3'])
            command.extend(['-i', deep2_labels[j]])
            command.extend(['-r', deep2_labels[i]])
            command.extend(['-o', output])
            command.extend(['-n', 'NearestNeighbor'])
            command.extend(['-t', deep2_bck_warps[i]])
            command.extend(['-t', deep2_fwd_warps[j]])
            subprocess.call(command)             
            deep2_jth_labels_to_ith_labels = torch.from_numpy(nib.load(output).get_fdata().ravel()).type('torch.LongTensor')
            deep2_values_ij = dicemetric(deep2_jth_labels_to_ith_labels,deep2_ith_labels)
            
            output = 'deep3_j_to_i_labels.nii.gz'
            command = ['antsApplyTransforms']
            command.extend(['-d', '3'])
            command.extend(['-i', deep3_labels[j]])
            command.extend(['-r', deep3_labels[i]])
            command.extend(['-o', output])
            command.extend(['-n', 'NearestNeighbor'])
            command.extend(['-t', deep3_bck_warps[i]])
            command.extend(['-t', deep3_fwd_warps[j]])
            subprocess.call(command)             
            deep3_jth_labels_to_ith_labels = torch.from_numpy(nib.load(output).get_fdata().ravel()).type('torch.LongTensor')
            deep3_values_ij = dicemetric(deep3_jth_labels_to_ith_labels,deep3_ith_labels)
            
            output = 'ants_j_to_i_labels.nii.gz'
            command = ['antsApplyTransforms']
            command.extend(['-d', '3'])
            command.extend(['-i', ants_labels[j]])
            command.extend(['-r', ants_labels[i]])
            command.extend(['-o', output])
            command.extend(['-n', 'NearestNeighbor'])
            command.extend(['-t', ants_bck_warps[i]])
            command.extend(['-t', ants_fwd_warps[j]])
            subprocess.call(command)             
            ants_jth_labels_to_ith_labels = torch.from_numpy(nib.load(output).get_fdata().ravel()).type('torch.LongTensor')
            ants_values_ij = dicemetric(ants_jth_labels_to_ith_labels,ants_ith_labels)
            
            output = 'mean_j_to_i_labels.nii.gz'
            command = ['antsApplyTransforms']
            command.extend(['-d', '3'])
            command.extend(['-i', mean_labels[j]])
            command.extend(['-r', mean_labels[i]])
            command.extend(['-o', output])
            command.extend(['-n', 'NearestNeighbor'])
            command.extend(['-t', mean_bck_warps[i]])
            command.extend(['-t', mean_fwd_warps[j]])
            subprocess.call(command)             
            mean_jth_labels_to_ith_labels = torch.from_numpy(nib.load(output).get_fdata().ravel()).type('torch.LongTensor')
            mean_values_ij = dicemetric(mean_jth_labels_to_ith_labels,mean_ith_labels)
            
            for k in range(len(deep1_values_ij)):
                values_affine[k] = affine_values_ij[k].item()
                values_deep1[k] = deep1_values_ij[k].item()
                values_deep2[k] = deep2_values_ij[k].item()
                values_deep3[k] = deep3_values_ij[k].item()
                values_ants[k] = ants_values_ij[k].item()
                values_mean[k] = mean_values_ij[k].item()
            name = str(i+1) + '_' + str(j+1)  
            affine_dict = {"affine_dice_" + str(region_names[k]): values_affine[k] for k in range(len(region_names))}
            ants_dict = {"ants_dice_" + str(region_names[k]): values_ants[k] for k in range(len(region_names))}
            deep1_dict = {"deep1_dice_" + str(region_names[k]): values_deep1[k] for k in range(len(region_names))}
            deep2_dict = {"deep2_dice_" + str(region_names[k]): values_deep2[k] for k in range(len(region_names))}
            deep3_dict = {"deep3_dice_" + str(region_names[k]): values_deep3[k] for k in range(len(region_names))}
            mean_dict = {"antsmean_dice_" + str(region_names[k]): values_mean[k] for k in range(len(region_names))}
            ith_dict = {"name": name}
            ith_dict.update(affine_dict)
            ith_dict.update(ants_dict)
            ith_dict.update(deep1_dict)
            ith_dict.update(deep2_dict)
            ith_dict.update(deep3_dict)
            ith_dict.update(mean_dict)
            list_dict.append(ith_dict)
print(ith_dict)
df = pd.DataFrame(list_dict)
df.to_csv("crossdice_self_results.csv")


{'name': '27_27', 'affine_dice_bg': 1.0, 'affine_dice_hy': 1.0, 'affine_dice_mb': 1.0, 'affine_dice_hpf': 1.0, 'affine_dice_str': 1.0, 'affine_dice_ctx': 1.0, 'affine_dice_gp': 1.0, 'affine_dice_th': 1.0, 'affine_dice_cb': 1.0, 'affine_dice_olf': 1.0, 'affine_dice_bs': 1.0, 'ants_dice_bg': 1.0, 'ants_dice_hy': 1.0, 'ants_dice_mb': 1.0, 'ants_dice_hpf': 1.0, 'ants_dice_str': 1.0, 'ants_dice_ctx': 1.0, 'ants_dice_gp': 1.0, 'ants_dice_th': 1.0, 'ants_dice_cb': 1.0, 'ants_dice_olf': 1.0, 'ants_dice_bs': 1.0, 'deep1_dice_bg': 0.9999856352806091, 'deep1_dice_hy': 1.0, 'deep1_dice_mb': 0.9998019337654114, 'deep1_dice_hpf': 0.999910295009613, 'deep1_dice_str': 1.0, 'deep1_dice_ctx': 0.9997577667236328, 'deep1_dice_gp': 1.0, 'deep1_dice_th': 0.9997677206993103, 'deep1_dice_cb': 0.9996752142906189, 'deep1_dice_olf': 1.0, 'deep1_dice_bs': 0.9994727969169617, 'deep2_dice_bg': 0.999993622303009, 'deep2_dice_hy': 1.0, 'deep2_dice_mb': 1.0, 'deep2_dice_hpf': 0.9995054006576538, 'deep2_dice_str': 1.0,

In [ ]:
atlas_mask = "dataset3/Atlas/P56_Annotation_128_norm_id_mask.nii.gz"
atlas_labels = "atlas_gin_map6.nii.gz"


affine_labels = sorted(glob(os.path.join('dataset3', 'GIN', 'Labels', "*_affine_convert.nii.gz")))

l10e3_labels = sorted(glob(os.path.join('output', 'GIN', 'lambda2_0.001', 'Labels', '*.nii.gz')))
l10e2_labels = sorted(glob(os.path.join('output', 'GIN', 'lambda2_0.01', 'Labels', '*.nii.gz')))
l10e1_labels = sorted(glob(os.path.join('output', 'GIN', 'lambda2_0.1', 'Labels', '*.nii.gz')))
l10e0_labels = sorted(glob(os.path.join('output', 'GIN', 'lambda2_1.0', 'Labels', '*.nii.gz')))
l10n1_labels = sorted(glob(os.path.join('output', 'GIN', 'lambda2_10', 'Labels', '*.nii.gz')))
l10n2_labels = sorted(glob(os.path.join('output', 'GIN', 'lambda2_100', 'Labels', '*.nii.gz')))
l10n3_labels = sorted(glob(os.path.join('output', 'GIN', 'lambda2_1000', 'Labels', '*.nii.gz')))

l10e3_fwd_warps = sorted(glob(os.path.join('output', 'GIN', 'lambda2_0.001', 'DeformableWarp', "*_warp.nii.gz")))
l10e2_fwd_warps = sorted(glob(os.path.join('output', 'GIN', 'lambda2_0.01', 'DeformableWarp', "*_warp.nii.gz")))
l10e1_fwd_warps = sorted(glob(os.path.join('output', 'GIN', 'lambda2_0.1', 'DeformableWarp', "*_warp.nii.gz")))
l10e0_fwd_warps = sorted(glob(os.path.join('output', 'GIN', 'lambda2_1.0', 'DeformableWarp', "*_warp.nii.gz")))
l10n1_fwd_warps = sorted(glob(os.path.join('output', 'GIN', 'lambda2_10', 'DeformableWarp', "*_warp.nii.gz")))
l10n2_fwd_warps = sorted(glob(os.path.join('output', 'GIN', 'lambda2_100', 'DeformableWarp', "*_warp.nii.gz")))
l10n3_fwd_warps = sorted(glob(os.path.join('output', 'GIN', 'lambda2_1000', 'DeformableWarp', "*_warp.nii.gz")))

l10e3_bck_warps = sorted(glob(os.path.join('output', 'GIN', 'lambda2_0.001', 'DeformableWarp', "*_invwarp.nii.gz")))
l10e2_bck_warps = sorted(glob(os.path.join('output', 'GIN', 'lambda2_0.01', 'DeformableWarp',  "*_invwarp.nii.gz")))
l10e1_bck_warps = sorted(glob(os.path.join('output', 'GIN', 'lambda2_0.1', 'DeformableWarp',   "*_invwarp.nii.gz")))
l10e0_bck_warps = sorted(glob(os.path.join('output', 'GIN', 'lambda2_1.0', 'DeformableWarp',   "*_invwarp.nii.gz")))
l10n1_bck_warps = sorted(glob(os.path.join('output', 'GIN', 'lambda2_10', 'DeformableWarp',    "*_invwarp.nii.gz")))
l10n2_bck_warps = sorted(glob(os.path.join('output', 'GIN', 'lambda2_100', 'DeformableWarp',   "*_invwarp.nii.gz")))
l10n3_bck_warps = sorted(glob(os.path.join('output', 'GIN', 'lambda2_1000', 'DeformableWarp',  "*_invwarp.nii.gz")))

dicemetric = torchmetrics.Dice(average=None, mdmc_average='global', num_classes=11)

values_affine = [0,0,0,0,0,0,0,0,0,0,0]
values_l10e3 = [0,0,0,0,0,0,0,0,0,0,0]
values_l10e2 = [0,0,0,0,0,0,0,0,0,0,0]
values_l10e1 = [0,0,0,0,0,0,0,0,0,0,0]
values_l10e0 = [0,0,0,0,0,0,0,0,0,0,0]
values_l10n1 = [0,0,0,0,0,0,0,0,0,0,0]
values_l10n2 = [0,0,0,0,0,0,0,0,0,0,0]
values_l10n3 = [0,0,0,0,0,0,0,0,0,0,0]

mapping_csv = "dataset3/GIN/labels_mapping.csv"
data = pd.read_csv(mapping_csv)
    
count = 0

list_dict = []

for i, _ in enumerate(l10e3_labels): 
    affine_ith_labels = torch.from_numpy(nib.load(affine_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
    l10e3_ith_labels = torch.from_numpy(nib.load(l10e3_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
    l10e2_ith_labels = torch.from_numpy(nib.load(l10e2_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
    l10e1_ith_labels = torch.from_numpy(nib.load(l10e1_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
    l10e0_ith_labels = torch.from_numpy(nib.load(l10e0_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
    l10n1_ith_labels = torch.from_numpy(nib.load(l10n1_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
    l10n2_ith_labels = torch.from_numpy(nib.load(l10n2_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
    l10n3_ith_labels = torch.from_numpy(nib.load(l10n3_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
    
    for j, _ in enumerate(l10e3_labels):
        
        if i != j:
            
            print(str(i+1) + '_' + str(j+1), end='\r')
            count += 1            

            affine_jth_labels = torch.from_numpy(nib.load(affine_labels[j]).get_fdata().ravel()).type('torch.LongTensor')
            affine_values_ij = dicemetric(affine_jth_labels,affine_ith_labels)

            output = 'temp_lambda1.nii.gz'
            command = ['antsApplyTransforms']
            command.extend(['-d', '3'])
            command.extend(['-i', l10e3_labels[j]])
            command.extend(['-r', l10e3_labels[i]])
            command.extend(['-o', output])
            command.extend(['-n', 'MultiLabel'])
            command.extend(['-t', l10e3_bck_warps[i]])
            command.extend(['-t', l10e3_fwd_warps[j]])
            subprocess.call(command)             
            l10e3_jth_labels_to_ith_labels = torch.from_numpy(nib.load(output).get_fdata().ravel()).type('torch.LongTensor')
            l10e3_values_ij = dicemetric(l10e3_jth_labels_to_ith_labels, l10e3_ith_labels)

            output = 'temp_lambda1.nii.gz'
            command = ['antsApplyTransforms']
            command.extend(['-d', '3'])
            command.extend(['-i', l10e2_labels[j]])
            command.extend(['-r', l10e2_labels[i]])
            command.extend(['-o', output])
            command.extend(['-n', 'MultiLabel'])
            command.extend(['-t', l10e2_bck_warps[i]])
            command.extend(['-t', l10e2_fwd_warps[j]])
            subprocess.call(command)             
            l10e2_jth_labels_to_ith_labels = torch.from_numpy(nib.load(output).get_fdata().ravel()).type('torch.LongTensor')
            l10e2_values_ij = dicemetric(l10e2_jth_labels_to_ith_labels, l10e2_ith_labels)

            output = 'temp_lambda1.nii.gz'
            command = ['antsApplyTransforms']
            command.extend(['-d', '3'])
            command.extend(['-i', l10e1_labels[j]])
            command.extend(['-r', l10e1_labels[i]])
            command.extend(['-o', output])
            command.extend(['-n', 'MultiLabel'])
            command.extend(['-t', l10e1_bck_warps[i]])
            command.extend(['-t', l10e1_fwd_warps[j]])
            subprocess.call(command)             
            l10e1_jth_labels_to_ith_labels = torch.from_numpy(nib.load(output).get_fdata().ravel()).type('torch.LongTensor')
            l10e1_values_ij = dicemetric(l10e1_jth_labels_to_ith_labels, l10e1_ith_labels)

            output = 'temp_lambda1.nii.gz'
            command = ['antsApplyTransforms']
            command.extend(['-d', '3'])
            command.extend(['-i', l10e0_labels[j]])
            command.extend(['-r', l10e0_labels[i]])
            command.extend(['-o', output])
            command.extend(['-n', 'MultiLabel'])
            command.extend(['-t', l10e0_bck_warps[i]])
            command.extend(['-t', l10e0_fwd_warps[j]])
            subprocess.call(command)             
            l10e0_jth_labels_to_ith_labels = torch.from_numpy(nib.load(output).get_fdata().ravel()).type('torch.LongTensor')
            l10e0_values_ij = dicemetric(l10e0_jth_labels_to_ith_labels, l10e0_ith_labels)

            output = 'temp_lambda1.nii.gz'
            command = ['antsApplyTransforms']
            command.extend(['-d', '3'])
            command.extend(['-i', l10n1_labels[j]])
            command.extend(['-r', l10n1_labels[i]])
            command.extend(['-o', output])
            command.extend(['-n', 'MultiLabel'])
            command.extend(['-t', l10n1_bck_warps[i]])
            command.extend(['-t', l10n1_fwd_warps[j]])
            subprocess.call(command)             
            l10n1_jth_labels_to_ith_labels = torch.from_numpy(nib.load(output).get_fdata().ravel()).type('torch.LongTensor')
            l10n1_values_ij = dicemetric(l10n1_jth_labels_to_ith_labels, l10n1_ith_labels)

            output = 'temp_lambda1.nii.gz'
            command = ['antsApplyTransforms']
            command.extend(['-d', '3'])
            command.extend(['-i', l10n2_labels[j]])
            command.extend(['-r', l10n2_labels[i]])
            command.extend(['-o', output])
            command.extend(['-n', 'MultiLabel'])
            command.extend(['-t', l10n2_bck_warps[i]])
            command.extend(['-t', l10n2_fwd_warps[j]])
            subprocess.call(command)             
            l10n2_jth_labels_to_ith_labels = torch.from_numpy(nib.load(output).get_fdata().ravel()).type('torch.LongTensor')
            l10n2_values_ij = dicemetric(l10n2_jth_labels_to_ith_labels, l10n2_ith_labels)

            output = 'temp_lambda1.nii.gz'
            command = ['antsApplyTransforms']
            command.extend(['-d', '3'])
            command.extend(['-i', l10n3_labels[j]])
            command.extend(['-r', l10n3_labels[i]])
            command.extend(['-o', output])
            command.extend(['-n', 'MultiLabel'])
            command.extend(['-t', l10n3_bck_warps[i]])
            command.extend(['-t', l10n3_fwd_warps[j]])
            subprocess.call(command)             
            l10n3_jth_labels_to_ith_labels = torch.from_numpy(nib.load(output).get_fdata().ravel()).type('torch.LongTensor')
            l10n3_values_ij = dicemetric(l10n3_jth_labels_to_ith_labels, l10n3_ith_labels)


            for k in range(len(l10e3_values_ij)):
                values_affine[k] = affine_values_ij[k].item()
                values_l10e3[k] = l10e3_values_ij[k].item()
                values_l10e2[k] = l10e2_values_ij[k].item()
                values_l10e1[k] = l10e1_values_ij[k].item()
                values_l10e0[k] = l10e0_values_ij[k].item()
                values_l10n1[k] = l10n1_values_ij[k].item()
                values_l10n2[k] = l10n2_values_ij[k].item()
                values_l10n3[k] = l10n3_values_ij[k].item()

            name = str(i+1) + '_' + str(j+1)  
            affine_dict = {"affine_dice_" + str(region_names[k]): values_affine[k] for k in range(len(region_names))}
            l10e3_dict = {"l10e3_dice_" + str(region_names[k]): values_l10e3[k] for k in range(len(region_names))}
            l10e2_dict = {"l10e2_dice_" + str(region_names[k]): values_l10e2[k] for k in range(len(region_names))}
            l10e1_dict = {"l10e1_dice_" + str(region_names[k]): values_l10e1[k] for k in range(len(region_names))}
            l10e0_dict = {"l10e0_dice_" + str(region_names[k]): values_l10e0[k] for k in range(len(region_names))}
            l10n1_dict = {"l10n1_dice_" + str(region_names[k]): values_l10n1[k] for k in range(len(region_names))}
            l10n2_dict = {"l10n2_dice_" + str(region_names[k]): values_l10n2[k] for k in range(len(region_names))}
            l10n3_dict = {"l10n3_dice_" + str(region_names[k]): values_l10n3[k] for k in range(len(region_names))}
            ith_dict = {"name": name}
            ith_dict.update(affine_dict)
            ith_dict.update(l10e3_dict)
            ith_dict.update(l10e2_dict)
            ith_dict.update(l10e1_dict)
            ith_dict.update(l10e0_dict)
            ith_dict.update(l10n1_dict)
            ith_dict.update(l10n2_dict)
            ith_dict.update(l10n3_dict)
            list_dict.append(ith_dict)
print(ith_dict)
df = pd.DataFrame(list_dict)
df.to_csv("lambda2_gin_interdice_results2.csv")


In [5]:
atlas_mask = "dataset3/Atlas/P56_Annotation_128_norm_id_mask.nii.gz"
atlas_labels = "atlas_gin_map6.nii.gz"

ith_labels = sorted(glob(os.path.join('output', 'GIN', 'ith', 'Labels', '*.nii.gz')))
ith2_labels = sorted(glob(os.path.join('output', 'GIN', 'ith2', 'Labels', '*.nii.gz')))
gin_labels = sorted(glob(os.path.join('output', 'GIN', 'lambda2_0.01', 'Labels', '*.nii.gz')))
all_labels = sorted(glob(os.path.join('output', 'GIN', 'gin+feminad', 'Labels', '*.nii.gz')))

ith_fwd_warps = sorted(glob(os.path.join('output', 'GIN', 'ith', 'DeformableWarp', "*_warp.nii.gz")))
ith2_fwd_warps = sorted(glob(os.path.join('output', 'GIN', 'ith2', 'DeformableWarp', "*_warp.nii.gz")))
gin_fwd_warps = sorted(glob(os.path.join('output', 'GIN', 'lambda2_0.01', 'DeformableWarp', "*_warp.nii.gz")))
all_fwd_warps = sorted(glob(os.path.join('output', 'GIN', 'gin+feminad', 'DeformableWarp', "*_warp.nii.gz")))

ith_bck_warps = sorted(glob(os.path.join('output', 'GIN', 'ith', 'DeformableWarp', "*_invwarp.nii.gz")))
ith2_bck_warps = sorted(glob(os.path.join('output', 'GIN', 'ith2', 'DeformableWarp', "*_invwarp.nii.gz")))
gin_bck_warps = sorted(glob(os.path.join('output', 'GIN', 'lambda2_0.01', 'DeformableWarp',  "*_invwarp.nii.gz")))
all_bck_warps = sorted(glob(os.path.join('output', 'GIN', 'gin+feminad', 'DeformableWarp',   "*_invwarp.nii.gz")))

dicemetric = torchmetrics.Dice(average=None, mdmc_average='global', num_classes=11)

#values_ith = [0,0,0,0,0,0,0,0,0,0,0]
values_ith2 = [0,0,0,0,0,0,0,0,0,0,0]
#values_gin = [0,0,0,0,0,0,0,0,0,0,0]
#values_all = [0,0,0,0,0,0,0,0,0,0,0]

mapping_csv = "dataset3/GIN/labels_mapping.csv"
data = pd.read_csv(mapping_csv)
    
count = 0

list_dict = []

for i, _ in enumerate(ith_labels): 
    
    #ith_ith_labels = torch.from_numpy(nib.load(ith_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
    ith2_ith_labels = torch.from_numpy(nib.load(ith2_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
    #gin_ith_labels = torch.from_numpy(nib.load(gin_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
    #all_ith_labels = torch.from_numpy(nib.load(all_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
    
    for j, _ in enumerate(ith_labels):
        
        if i != j:
            
            print(str(i+1) + '_' + str(j+1), end='\r')
            count += 1            

            #output = 'temp_lambda1.nii.gz'
            #command = ['antsApplyTransforms']
            #command.extend(['-d', '3'])
            #command.extend(['-i', ith_labels[j]])
            #command.extend(['-r', ith_labels[i]])
            #command.extend(['-o', output])
            #command.extend(['-n', 'NearestNeighbor'])
            #command.extend(['-t', ith_bck_warps[i]])
            #command.extend(['-t', ith_fwd_warps[j]])
            #subprocess.call(command)             
            #ith_jth_labels_to_ith_labels = torch.from_numpy(nib.load(output).get_fdata().ravel()).type('torch.LongTensor')
            #ith_values_ij = dicemetric(ith_jth_labels_to_ith_labels, ith_ith_labels)
            
            #output = 'temp_lambda1.nii.gz'
            #command = ['antsApplyTransforms']
            #command.extend(['-d', '3'])
            #command.extend(['-i', gin_labels[j]])
            #command.extend(['-r', gin_labels[i]])
            #command.extend(['-o', output])
            #command.extend(['-n', 'NearestNeighbor'])
            #command.extend(['-t', gin_bck_warps[i]])
            #command.extend(['-t', gin_fwd_warps[j]])
            #subprocess.call(command)             
            #gin_jth_labels_to_ith_labels = torch.from_numpy(nib.load(output).get_fdata().ravel()).type('torch.LongTensor')
            #gin_values_ij = dicemetric(gin_jth_labels_to_ith_labels, gin_ith_labels)
            
            #output = 'temp_lambda1.nii.gz'
            #command = ['antsApplyTransforms']
            #command.extend(['-d', '3'])
            #command.extend(['-i', all_labels[j]])
            #command.extend(['-r', all_labels[i]])
            #command.extend(['-o', output])
            #command.extend(['-n', 'NearestNeighbor'])
            #command.extend(['-t', all_bck_warps[i]])
            #command.extend(['-t', all_fwd_warps[j]])
            #subprocess.call(command)             
            #all_jth_labels_to_ith_labels = torch.from_numpy(nib.load(output).get_fdata().ravel()).type('torch.LongTensor')
            #all_values_ij = dicemetric(all_jth_labels_to_ith_labels, all_ith_labels)
            
            output = 'temp_lambda1.nii.gz'
            command = ['antsApplyTransforms']
            command.extend(['-d', '3'])
            command.extend(['-i', ith2_labels[j]])
            command.extend(['-r', ith2_labels[i]])
            command.extend(['-o', output])
            command.extend(['-n', 'NearestNeighbor'])
            command.extend(['-t', ith2_bck_warps[i]])
            command.extend(['-t', ith2_fwd_warps[j]])
            subprocess.call(command)             
            ith2_jth_labels_to_ith_labels = torch.from_numpy(nib.load(output).get_fdata().ravel()).type('torch.LongTensor')
            ith2_values_ij = dicemetric(ith2_jth_labels_to_ith_labels, ith2_ith_labels)


            for k in range(len(ith_values_ij)):
                values_ith[k] = ith_values_ij[k].item()
                values_ith2[k] = ith2_values_ij[k].item()
                values_gin[k] = gin_values_ij[k].item()
                values_all[k] = all_values_ij[k].item()

            name = str(i+1) + '_' + str(j+1)  
            #vith_dict = {"final_ith_" + str(region_names[k]): values_ith[k] for k in range(len(region_names))}
            vith2_dict = {"final_ith_noaug" + str(region_names[k]): values_ith2[k] for k in range(len(region_names))}
            #gin_dict = {"final_gin_" + str(region_names[k]): values_gin[k] for k in range(len(region_names))}
            #all_dict = {"final_all_" + str(region_names[k]): values_all[k] for k in range(len(region_names))}
            ith_dict = {"name": name}
            #ith_dict.update(vith_dict)
            ith_dict.update(vith2_dict)
            #ith_dict.update(gin_dict)
            #ith_dict.update(all_dict)
            list_dict.append(ith_dict)
print(ith_dict)
df = pd.DataFrame(list_dict)
df.to_csv("final_interdice2_results.csv")


{'name': '27_26', 'final_ith_noaugbg': 0.9833449125289917, 'final_ith_noaughy': 0.8410511016845703, 'final_ith_noaugmb': 0.794305145740509, 'final_ith_noaughpf': 0.7878218293190002, 'final_ith_noaugstr': 0.7560827732086182, 'final_ith_noaugctx': 0.7967846393585205, 'final_ith_noauggp': 0.26565051078796387, 'final_ith_noaugth': 0.8393359780311584, 'final_ith_noaugcb': 0.8520523309707642, 'final_ith_noaugolf': 0.7870007157325745, 'final_ith_noaugbs': 0.8346587419509888}


In [ ]:
atlas_mask = "dataset3/Atlas/P56_Annotation_128_norm_id_mask.nii.gz"
atlas_labels = "atlas_gin_map6.nii.gz"

ith_labels = sorted(glob(os.path.join('output', 'GIN', 'ith', 'Labels', '*.nii.gz')))
ith2_labels = sorted(glob(os.path.join('output', 'GIN', 'ith2', 'Labels', '*.nii.gz')))
ith3_labels = sorted(glob(os.path.join('output', 'GIN', 'ith3', 'Labels', '*.nii.gz')))
gin_labels = sorted(glob(os.path.join('output', 'GIN', 'lambda2_0.01', 'Labels', '*.nii.gz')))
all_labels = sorted(glob(os.path.join('output', 'GIN', 'gin+feminad', 'Labels', '*.nii.gz')))

ith_fwd_warps = sorted(glob(os.path.join('output', 'GIN', 'ith', 'DeformableWarp', "*_warp.nii.gz")))
ith2_fwd_warps = sorted(glob(os.path.join('output', 'GIN', 'ith2', 'DeformableWarp', "*_warp.nii.gz")))
ith3_fwd_warps = sorted(glob(os.path.join('output', 'GIN', 'ith3', 'DeformableWarp', "*_warp.nii.gz")))
gin_fwd_warps = sorted(glob(os.path.join('output', 'GIN', 'lambda2_0.01', 'DeformableWarp', "*_warp.nii.gz")))
all_fwd_warps = sorted(glob(os.path.join('output', 'GIN', 'gin+feminad', 'DeformableWarp', "*_warp.nii.gz")))

ith_bck_warps = sorted(glob(os.path.join('output', 'GIN', 'ith', 'DeformableWarp', "*_invwarp.nii.gz")))
ith2_bck_warps = sorted(glob(os.path.join('output', 'GIN', 'ith2', 'DeformableWarp', "*_invwarp.nii.gz")))
ith3_bck_warps = sorted(glob(os.path.join('output', 'GIN', 'ith3', 'DeformableWarp', "*_invwarp.nii.gz")))
gin_bck_warps = sorted(glob(os.path.join('output', 'GIN', 'lambda2_0.01', 'DeformableWarp',  "*_invwarp.nii.gz")))
all_bck_warps = sorted(glob(os.path.join('output', 'GIN', 'gin+feminad', 'DeformableWarp',   "*_invwarp.nii.gz")))

dicemetric = torchmetrics.Dice(average=None, mdmc_average='global', num_classes=11)

values_ith = [0,0,0,0,0,0,0,0,0,0,0]
values_ith2 = [0,0,0,0,0,0,0,0,0,0,0]
values_ith3 = [0,0,0,0,0,0,0,0,0,0,0]
values_gin = [0,0,0,0,0,0,0,0,0,0,0]
values_all = [0,0,0,0,0,0,0,0,0,0,0]

mapping_csv = "dataset3/GIN/labels_mapping.csv"
data = pd.read_csv(mapping_csv)
    
count = 0

list_dict = []

for i, _ in enumerate(ith_labels): 
    
    ith_ith_labels = torch.from_numpy(nib.load(ith_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
    ith2_ith_labels = torch.from_numpy(nib.load(ith2_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
    ith3_ith_labels = torch.from_numpy(nib.load(ith3_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
    gin_ith_labels = torch.from_numpy(nib.load(gin_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
    all_ith_labels = torch.from_numpy(nib.load(all_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
    
    for j, _ in enumerate(ith_labels):
        
        if i == j:
            
            print(str(i+1) + '_' + str(j+1), end='\r')
            count += 1            

            output = 'temp_lambda1.nii.gz'
            command = ['antsApplyTransforms']
            command.extend(['-d', '3'])
            command.extend(['-i', ith_labels[j]])
            command.extend(['-r', ith_labels[i]])
            command.extend(['-o', output])
            command.extend(['-n', 'NearestNeighbor'])
            command.extend(['-t', ith_bck_warps[i]])
            command.extend(['-t', ith_fwd_warps[j]])
            subprocess.call(command)             
            ith_jth_labels_to_ith_labels = torch.from_numpy(nib.load(output).get_fdata().ravel()).type('torch.LongTensor')
            ith_values_ij = dicemetric(ith_jth_labels_to_ith_labels, ith_ith_labels)
            
            output = 'temp_lambda1.nii.gz'
            command = ['antsApplyTransforms']
            command.extend(['-d', '3'])
            command.extend(['-i', ith2_labels[j]])
            command.extend(['-r', ith2_labels[i]])
            command.extend(['-o', output])
            command.extend(['-n', 'NearestNeighbor'])
            command.extend(['-t', ith2_bck_warps[i]])
            command.extend(['-t', ith2_fwd_warps[j]])
            subprocess.call(command)             
            ith2_jth_labels_to_ith_labels = torch.from_numpy(nib.load(output).get_fdata().ravel()).type('torch.LongTensor')
            ith2_values_ij = dicemetric(ith2_jth_labels_to_ith_labels, ith2_ith_labels)
            
            output = 'temp_lambda1.nii.gz'
            command = ['antsApplyTransforms']
            command.extend(['-d', '3'])
            command.extend(['-i', ith3_labels[j]])
            command.extend(['-r', ith3_labels[i]])
            command.extend(['-o', output])
            command.extend(['-n', 'NearestNeighbor'])
            command.extend(['-t', ith3_bck_warps[i]])
            command.extend(['-t', ith3_fwd_warps[j]])
            subprocess.call(command)             
            ith3_jth_labels_to_ith_labels = torch.from_numpy(nib.load(output).get_fdata().ravel()).type('torch.LongTensor')
            ith3_values_ij = dicemetric(ith3_jth_labels_to_ith_labels, ith3_ith_labels)

            output = 'temp_lambda1.nii.gz'
            command = ['antsApplyTransforms']
            command.extend(['-d', '3'])
            command.extend(['-i', gin_labels[j]])
            command.extend(['-r', gin_labels[i]])
            command.extend(['-o', output])
            command.extend(['-n', 'NearestNeighbor'])
            command.extend(['-t', gin_bck_warps[i]])
            command.extend(['-t', gin_fwd_warps[j]])
            subprocess.call(command)             
            gin_jth_labels_to_ith_labels = torch.from_numpy(nib.load(output).get_fdata().ravel()).type('torch.LongTensor')
            gin_values_ij = dicemetric(gin_jth_labels_to_ith_labels, gin_ith_labels)

            output = 'temp_lambda1.nii.gz'
            command = ['antsApplyTransforms']
            command.extend(['-d', '3'])
            command.extend(['-i', all_labels[j]])
            command.extend(['-r', all_labels[i]])
            command.extend(['-o', output])
            command.extend(['-n', 'NearestNeighbor'])
            command.extend(['-t', all_bck_warps[i]])
            command.extend(['-t', all_fwd_warps[j]])
            subprocess.call(command)             
            all_jth_labels_to_ith_labels = torch.from_numpy(nib.load(output).get_fdata().ravel()).type('torch.LongTensor')
            all_values_ij = dicemetric(all_jth_labels_to_ith_labels, all_ith_labels)


            for k in range(len(ith_values_ij)):
                values_ith[k] = ith_values_ij[k].item()
                values_ith2[k] = ith2_values_ij[k].item()
                values_ith3[k] = ith3_values_ij[k].item()
                values_gin[k] = gin_values_ij[k].item()
                values_all[k] = all_values_ij[k].item()

            name = str(i+1) + '_' + str(j+1)  
            vith_dict = {"final_ith_aug_" + str(region_names[k]): values_ith[k] for k in range(len(region_names))}
            vith2_dict = {"final_ith_noaug_" + str(region_names[k]): values_ith2[k] for k in range(len(region_names))}
            vith3_dict = {"final_ith_overfit_" + str(region_names[k]): values_ith3[k] for k in range(len(region_names))}
            gin_dict = {"final_gin_" + str(region_names[k]): values_gin[k] for k in range(len(region_names))}
            all_dict = {"final_all_" + str(region_names[k]): values_all[k] for k in range(len(region_names))}
            ith_dict = {"name": name}
            ith_dict.update(vith_dict)
            ith_dict.update(vith2_dict)
            ith_dict.update(vith3_dict)
            ith_dict.update(gin_dict)
            ith_dict.update(all_dict)
            list_dict.append(ith_dict)
print(ith_dict)
df = pd.DataFrame(list_dict)
df.to_csv("final_interdice3_self_results.csv")


In [5]:
atlas_mask = "dataset3/Atlas/P56_Annotation_128_norm_id_mask.nii.gz"
atlas_labels = "atlas_gin_map6.nii.gz"

gin_labels = sorted(glob(os.path.join('output', 'GIN', 'lambda2_0.01_noaug', 'Labels', '*.nii.gz')))
gin_fwd_warps = sorted(glob(os.path.join('output', 'GIN', 'lambda2_0.01_noaug', 'DeformableWarp', "*_warp.nii.gz")))
gin_bck_warps = sorted(glob(os.path.join('output', 'GIN', 'lambda2_0.01_noaug', 'DeformableWarp',  "*_invwarp.nii.gz")))

dicemetric = torchmetrics.Dice(average=None, mdmc_average='global', num_classes=11)

values_gin = [0,0,0,0,0,0,0,0,0,0,0]

mapping_csv = "dataset3/GIN/labels_mapping.csv"
data = pd.read_csv(mapping_csv)
    
count = 0

list_dict = []

for i, _ in enumerate(ith_labels): 
    
    gin_ith_labels = torch.from_numpy(nib.load(gin_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
    
    for j, _ in enumerate(ith_labels):
        
        if i != j:
            
            print(str(i+1) + '_' + str(j+1), end='\r')
            count += 1            

            output = 'temp_lambda1.nii.gz'
            command = ['antsApplyTransforms']
            command.extend(['-d', '3'])
            command.extend(['-i', gin_labels[j]])
            command.extend(['-r', gin_labels[i]])
            command.extend(['-o', output])
            command.extend(['-n', 'NearestNeighbor'])
            command.extend(['-t', gin_bck_warps[i]])
            command.extend(['-t', gin_fwd_warps[j]])
            subprocess.call(command)             
            gin_jth_labels_to_ith_labels = torch.from_numpy(nib.load(output).get_fdata().ravel()).type('torch.LongTensor')
            gin_values_ij = dicemetric(gin_jth_labels_to_ith_labels, gin_ith_labels)


            for k in range(len(ith_values_ij)):
                values_gin[k] = gin_values_ij[k].item()

            name = str(i+1) + '_' + str(j+1)  
            gin_dict = {"final_gin_noaug_" + str(region_names[k]): values_gin[k] for k in range(len(region_names))}
            ith_dict = {"name": name}
            ith_dict.update(gin_dict)
            list_dict.append(ith_dict)
print(ith_dict)
df = pd.DataFrame(list_dict)
df.to_csv("final_interdice4__results.csv")


{'name': '27_26', 'final_gin_noaug_bg': 0.9871131777763367, 'final_gin_noaug_hy': 0.8261575698852539, 'final_gin_noaug_mb': 0.8540350794792175, 'final_gin_noaug_hpf': 0.9074149131774902, 'final_gin_noaug_str': 0.888947069644928, 'final_gin_noaug_ctx': 0.8755062818527222, 'final_gin_noaug_gp': 0.7273631691932678, 'final_gin_noaug_th': 0.9028486013412476, 'final_gin_noaug_cb': 0.8719596266746521, 'final_gin_noaug_olf': 0.8549700379371643, 'final_gin_noaug_bs': 0.8457802534103394}


In [6]:
atlas_mask = "dataset3/Atlas/P56_Annotation_128_norm_id_mask.nii.gz"
atlas_labels = "atlas_gin_map6.nii.gz"

ith2_labels = sorted(glob(os.path.join('output', 'GIN', 'dl-iris-sym', 'Labels', '*.nii.gz')))
ith3_labels = sorted(glob(os.path.join('output', 'GIN', 'dl-iris+op-sym', 'Labels', '*.nii.gz')))

ith2_fwd_warps = sorted(glob(os.path.join('output', 'GIN', 'dl-iris-sym', 'DeformableWarp', "*_warp.nii.gz")))
ith3_fwd_warps = sorted(glob(os.path.join('output', 'GIN', 'dl-iris+op-sym', 'DeformableWarp', "*_warp.nii.gz")))

ith2_bck_warps = sorted(glob(os.path.join('output', 'GIN', 'dl-iris-sym', 'DeformableWarp', "*_invwarp.nii.gz")))
ith3_bck_warps = sorted(glob(os.path.join('output', 'GIN', 'dl-iris+op-sym', 'DeformableWarp', "*_invwarp.nii.gz")))

dicemetric = torchmetrics.Dice(average=None, mdmc_average='global', num_classes=11)

values_ith2 = [0,0,0,0,0,0,0,0,0,0,0]
values_ith3 = [0,0,0,0,0,0,0,0,0,0,0]
mapping_csv = "dataset3/GIN/labels_mapping.csv"
data = pd.read_csv(mapping_csv)
    
count = 0

list_dict = []

for i, _ in enumerate(ith2_labels): 
    
    ith2_ith_labels = torch.from_numpy(nib.load(ith2_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
    ith3_ith_labels = torch.from_numpy(nib.load(ith3_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
    
    for j, _ in enumerate(ith2_labels):
        
        if i != j:
            
            print(str(i+1) + '_' + str(j+1), end='\r')
            count += 1            

            output = 'temp_lambda1.nii.gz'
            command = ['antsApplyTransforms']
            command.extend(['-d', '3'])
            command.extend(['-i', ith2_labels[j]])
            command.extend(['-r', ith2_labels[i]])
            command.extend(['-o', output])
            command.extend(['-n', 'NearestNeighbor'])
            command.extend(['-t', ith2_bck_warps[i]])
            command.extend(['-t', ith2_fwd_warps[j]])
            subprocess.call(command)             
            ith2_jth_labels_to_ith_labels = torch.from_numpy(nib.load(output).get_fdata().ravel()).type('torch.LongTensor')
            ith2_values_ij = dicemetric(ith2_jth_labels_to_ith_labels, ith2_ith_labels)
            
            output = 'temp_lambda1.nii.gz'
            command = ['antsApplyTransforms']
            command.extend(['-d', '3'])
            command.extend(['-i', ith3_labels[j]])
            command.extend(['-r', ith3_labels[i]])
            command.extend(['-o', output])
            command.extend(['-n', 'NearestNeighbor'])
            command.extend(['-t', ith3_bck_warps[i]])
            command.extend(['-t', ith3_fwd_warps[j]])
            subprocess.call(command)             
            ith3_jth_labels_to_ith_labels = torch.from_numpy(nib.load(output).get_fdata().ravel()).type('torch.LongTensor')
            ith3_values_ij = dicemetric(ith3_jth_labels_to_ith_labels, ith3_ith_labels)


            for k in range(len(ith2_values_ij)):
                values_ith2[k] = ith2_values_ij[k].item()
                values_ith3[k] = ith3_values_ij[k].item()

            name = str(i+1) + '_' + str(j+1)  
            vith2_dict = {"final_ith_noaug_" + str(region_names[k]): values_ith2[k] for k in range(len(region_names))}
            vith3_dict = {"final_ith_overfit_" + str(region_names[k]): values_ith3[k] for k in range(len(region_names))}
            ith_dict = {"name": name}
            ith_dict.update(vith2_dict)
            ith_dict.update(vith3_dict)
            list_dict.append(ith_dict)
print(ith_dict)
df = pd.DataFrame(list_dict)
df.to_csv("final_iris_ginsym_results.csv")


{'name': '27_26', 'final_ith_noaug_bg': 0.991762101650238, 'final_ith_noaug_hy': 0.8340438604354858, 'final_ith_noaug_mb': 0.8923565745353699, 'final_ith_noaug_hpf': 0.9249651432037354, 'final_ith_noaug_str': 0.9088298082351685, 'final_ith_noaug_ctx': 0.927435576915741, 'final_ith_noaug_gp': 0.7159624695777893, 'final_ith_noaug_th': 0.9202224612236023, 'final_ith_noaug_cb': 0.932740330696106, 'final_ith_noaug_olf': 0.8721779584884644, 'final_ith_noaug_bs': 0.887941300868988, 'final_ith_overfit_bg': 0.9918006658554077, 'final_ith_overfit_hy': 0.836562991142273, 'final_ith_overfit_mb': 0.9040172100067139, 'final_ith_overfit_hpf': 0.9205461144447327, 'final_ith_overfit_str': 0.9099674820899963, 'final_ith_overfit_ctx': 0.9272486567497253, 'final_ith_overfit_gp': 0.7218934893608093, 'final_ith_overfit_th': 0.9205479621887207, 'final_ith_overfit_cb': 0.930435299873352, 'final_ith_overfit_olf': 0.8773078918457031, 'final_ith_overfit_bs': 0.8696842193603516}


In [3]:
atlas_mask = "dataset3/Atlas/P56_Annotation_128_norm_id_mask.nii.gz"
atlas_labels = "atlas_gin_map6.nii.gz"

ith2_labels = sorted(glob(os.path.join('output', 'GIN', 'gin+iris', 'Labels', '*.nii.gz')))
ith2_fwd_warps = sorted(glob(os.path.join('output', 'GIN', 'gin+iris', 'DeformableWarp', "*_warp.nii.gz")))
ith2_bck_warps = sorted(glob(os.path.join('output', 'GIN', 'gin+iris', 'DeformableWarp', "*_invwarp.nii.gz")))

dicemetric = torchmetrics.Dice(average=None, mdmc_average='global', num_classes=11)

values_ith2 = [0,0,0,0,0,0,0,0,0,0,0]
mapping_csv = "dataset3/GIN/labels_mapping.csv"
data = pd.read_csv(mapping_csv)
    
count = 0

list_dict = []

for i, _ in enumerate(ith2_labels): 
    
    ith2_ith_labels = torch.from_numpy(nib.load(ith2_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
    
    for j, _ in enumerate(ith2_labels):
        
        if i != j:
            
            print(str(i+1) + '_' + str(j+1), end='\r')
            count += 1            

            output = 'temp_lambda1.nii.gz'
            command = ['antsApplyTransforms']
            command.extend(['-d', '3'])
            command.extend(['-i', ith2_labels[j]])
            command.extend(['-r', ith2_labels[i]])
            command.extend(['-o', output])
            command.extend(['-n', 'NearestNeighbor'])
            command.extend(['-t', ith2_bck_warps[i]])
            command.extend(['-t', ith2_fwd_warps[j]])
            subprocess.call(command)             
            ith2_jth_labels_to_ith_labels = torch.from_numpy(nib.load(output).get_fdata().ravel()).type('torch.LongTensor')
            ith2_values_ij = dicemetric(ith2_jth_labels_to_ith_labels, ith2_ith_labels)
            

            for k in range(len(ith2_values_ij)):
                values_ith2[k] = ith2_values_ij[k].item()

            name = str(i+1) + '_' + str(j+1)  
            vith2_dict = {"gin+iris_" + str(region_names[k]): values_ith2[k] for k in range(len(region_names))}
            ith_dict = {"name": name}
            ith_dict.update(vith2_dict)
            list_dict.append(ith_dict)
print(ith_dict)
df = pd.DataFrame(list_dict)
df.to_csv("gin+iris_crossdice_results.csv")


{'name': '27_26', 'gin+iris_bg': 0.9889909029006958, 'gin+iris_hy': 0.8317636847496033, 'gin+iris_mb': 0.8881163597106934, 'gin+iris_hpf': 0.9051545858383179, 'gin+iris_str': 0.8914276957511902, 'gin+iris_ctx': 0.8728063702583313, 'gin+iris_gp': 0.6674195528030396, 'gin+iris_th': 0.9073901772499084, 'gin+iris_cb': 0.9168821573257446, 'gin+iris_olf': 0.8586300611495972, 'gin+iris_bs': 0.8786551356315613}
